### Решение лабы 04

In [17]:
# импортируем необходимые пакеты
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

In [18]:
# просмотр всех колонок без многоточий
pd.set_option('display.max_columns', 120)

In [19]:
# чтение файла в DataFrame
df = pd.read_csv("/data/share/lab04data/lab04_train.csv", index_col=0)

In [20]:
# удаляем строки с пустым значением в поле TARGET
df = df[(~df['TARGET'].isnull())]

In [23]:
# заполняем пустые значения нулями
df = df.fillna(0)

In [26]:
# список столбцов с категорийными значениями
df.select_dtypes(['object']).columns.tolist()

['CLNT_TRUST_RELATION',
 'APP_MARITAL_STATUS',
 'APP_KIND_OF_PROP_HABITATION',
 'CLNT_JOB_POSITION_TYPE',
 'CLNT_JOB_POSITION',
 'APP_DRIVING_LICENSE',
 'APP_EDUCATION',
 'APP_TRAVEL_PASS',
 'APP_CAR',
 'APP_POSITION_TYPE',
 'APP_EMP_TYPE',
 'APP_COMP_TYPE',
 'PACK']

In [27]:
# удаляем колонки с категорийными значениями
df.drop(df.select_dtypes(['object']).columns.tolist(), axis=1, inplace=True)

In [28]:
# формируем датасет для обучения
df_train = df.drop('TARGET', axis=1)
# формируем target для обучения
y_train = df['TARGET']

In [29]:
# обучаем модель
model = XGBClassifier()
model.fit(df_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [30]:
# загружаем тестовый датасет
df_test = pd.read_csv("/data/share/lab04data/lab04_test.csv", index_col=0)

In [31]:
# приводим структура тестового датасета к структуре тренировочного (удаляем категорийные столбцы)
df_test.drop(df_test.select_dtypes(['object']).columns.tolist(), axis=1, inplace=True)

In [32]:
# формируем результат предсказания на нашей модели
pred = pd.Series(model.predict_proba(df_test)[:, 1])

In [33]:
# соединяем тестовый датасет и результат предсказания
df_res = df_test.assign(TARGET=pred.values)

In [34]:
# подготовка к формированию результирующего файла
df_result=df_res[['ID','TARGET']].rename({'ID':'id','TARGET':'target'}, axis=1)

In [36]:
# выгрузка результатов в файл в домашнюю директорию
df_result.to_csv('lab04.csv', sep='\t', index=False, header = True)